In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import json

In [ ]:
# merge all "x_monthly_allmodels.nc" together

In [2]:
# Open the datasets
r = xr.open_dataset("Africa_PearsonR_monthly_allmodels.nc")
rmse = xr.open_dataset("Africa_RMSE_monthly_allmodels.nc")
mae = xr.open_dataset("Africa_MAE_monthly_allmodels.nc")
mbe = xr.open_dataset("Africa_MBE_monthly_allmodels.nc")

# List of shared variables
variables = list(set(r.data_vars) & set(rmse.data_vars) & set(mae.data_vars) & set(mbe.data_vars))

# Build a new dataset with a 'metric' dimension
combined_vars = {}
for var in variables:
    combined_vars[var] = xr.concat(
        [r[var].expand_dims(metric=["r"]),
         rmse[var].expand_dims(metric=["rmse"]),
         mae[var].expand_dims(metric=["mae"]),
         mbe[var].expand_dims(metric=["mbe"]),
        ],
        dim="metric"
    )

# Combine all variables into one Dataset
combined = xr.Dataset(combined_vars)

combined.to_netcdf("Africa_R_RMSE_MAE_MBE_monthly_allmodels.nc")

# Final structure
print(combined)

<xarray.Dataset> Size: 6kB
Dimensions:      (metric: 4, month: 13, model: 3)
Coordinates:
  * metric       (metric) object 32B 'r' 'rmse' 'mae' 'mbe'
  * month        (month) int64 104B 1 2 3 4 5 6 7 8 9 10 11 12 13
  * model        (model) <U6 72B 'marsai' 'gc' 'marsfc'
    spatial_ref  int64 8B 0
Data variables:
    v10          (metric, model, month) float64 1kB nan nan ... -0.04144
    u10          (metric, model, month) float64 1kB nan nan ... 0.08843 0.003803
    t2m          (metric, model, month) float64 1kB nan nan ... -0.5321 -0.3152
    msl          (metric, model, month) float64 1kB nan nan ... -0.3409 -0.3692
    q            (metric, model, month) float64 1kB nan nan ... 0.06168 0.0699


In [ ]:
# turn the "R_RMSE_MAE_MBE_monthly_allmodels.nc" into a json file

In [3]:
# Load dataset
ds = xr.open_dataset("Africa_R_RMSE_MAE_MBE_monthly_allmodels.nc")
# Drop the 'number' coordinate if it exists
if "number" in ds.coords:
    ds = ds.drop_vars("number")

# Convert to xarray.Dataset → pandas.DataFrame
df = ds.to_dataframe().reset_index()


# Convert DataFrame to a list of dictionaries (good for plotting)
data = df.to_dict(orient="records")

# Save to JSON
with open("Africa_R_RMSE_MAE_MBE_monthly_allmodels.json", "w") as f:
    json.dump(data, f, indent=2)

print("Saved as Africa_R_RMSE_MAE_MBE_monthly_allmodels.json")

Saved as Africa_R_RMSE_MAE_MBE_monthly_allmodels.json
